In [88]:
import numpy as np
import shutil

import os


import SimpleITK as sitk
import nibabel as nib
import glob

In [40]:
image_l[0]

'/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0001_0000.nii.gz'

In [75]:


img_nii=nib.load(image_l[0])
img_data = img_nii.get_fdata()[None]
data_npy = np.vstack([d.get_fdata()] for d in [img_nii])
data_npy.shape

/var/folders/b_/lgrfxxx575sg8v5fpdzxfmtm0000gn/T/ipykernel_82066/1191952432.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data_npy = np.vstack([d.get_fdata()] for d in [img_nii])


(1, 512, 512, 110)

In [45]:
data_files=[image_l[0]]

In [85]:
d=sitk.ReadImage(image_l[0])
s=sitk.ReadImage(mask_l[0])
s_np=sitk.GetArrayFromImage(s)[None]



np.array(d.GetSize())[[2,1,0]].shape

(3,)

In [90]:
!pwd

/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data


In [95]:
data_itks=[sitk.ReadImage('/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Task501_FLARE/imagesTr/FLARE_002_0000.nii.gz')]



In [57]:
np.vstack([sitk.GetArrayFromImage(d_)[None] for d_ in [d]]).shape

(1, 110, 512, 512)

In [94]:
data_nib = [nib.load(f) for f in data_files]

header = data_nib[0].header
affine = data_nib[0].affine

properties = {}
properties["original_size_of_raw_data"] = np.array(header.get_data_shape())[[2, 1, 0]]
properties["original_spacing"] = np.array(header.get_zooms())[[2, 1, 0]]
properties["list_of_data_files"] = data_files
properties["seg_file"] = 'seg_file.nii.gz'  # replace with your actual segmentation file path

# NiBabel does not provide direct methods for origin and direction
# However, you can extract the affine transformation matrix components

properties["itk_origin"] = affine[:3, 3]
properties["itk_spacing"] = np.linalg.norm(affine[:3, :3], axis=0)
properties["itk_direction"] = affine[:3, :3] / properties["itk_spacing"]



for i in properties.items():
    print(i)

TypeError: expected str, bytes or os.PathLike object, not Image

[<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x13b71de30> >]

In [103]:
data_itk=[sitk.ReadImage('/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Task501_FLARE/imagesTr/FLARE_002_0000.nii.gz')]


properties["original_size_of_raw_data"] = np.array(data_itk[0].GetSize())[[2, 1, 0]]
properties["original_spacing"] = np.array(data_itk[0].GetSpacing())[[2, 1, 0]]
properties["list_of_data_files"] = data_files
properties["seg_file"] = 'seg_file.nii.gz'

properties["itk_origin"] = data_itk[0].GetOrigin()
properties["itk_spacing"] = data_itk[0].GetSpacing()
properties["itk_direction"] = data_itk[0].GetDirection()

data_npy = np.vstack([sitk.GetArrayFromImage(d)[None] for d in data_itk])
seg_file=""
if seg_file is not None:
    seg_itk = sitk.ReadImage("/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Task501_FLARE/labelsTr/FLARE_002.nii.gz")
    seg_npy = sitk.GetArrayFromImage(seg_itk)[None].astype(np.float32)
else:
    seg_npy = None
data_npy.astype(np.float32).shape, seg_npy.shape,properties

((1, 110, 512, 512),
 (1, 92, 512, 512),
 {'original_size_of_raw_data': array([110, 512, 512]),
  'original_spacing': array([2.5       , 0.64453101, 0.64453101]),
  'list_of_data_files': '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0001_0000.nii.gz',
  'seg_file': 'seg_file.nii.gz',
  'itk_origin': (329.3553466796875, 329.3553466796875, 0.0),
  'itk_spacing': (0.6445310115814209, 0.6445310115814209, 2.5),
  'itk_direction': (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)})

((1, 110, 512, 512),
 (1, 92, 512, 512),
 {'original_size_of_raw_data': array([110, 512, 512]),
  'original_spacing': array([2.5       , 0.64453101, 0.64453101]),
  'list_of_data_files': '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0001_0000.nii.gz',
  'seg_file': 'seg_file.nii.gz',
  'itk_origin': (329.3553466796875, 329.3553466796875, 0.0),
  'itk_spacing': (0.6445310115814209, 0.6445310115814209, 2.5),
  'itk_direction': (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)})

In [23]:
image_list=sorted(glob.glob("/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Dataset500_FLARE/imagesTr/*.nii.gz"))
label_list=sorted(glob.glob("/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Dataset500_FLARE/labelsTr/*.nii.gz"))
for image_file, label_file in zip(image_list, label_list):
    image = sitk.ReadImage(image_file)
    label = sitk.ReadImage(label_file)

    image_array = sitk.GetArrayFromImage(image)
    label_array = sitk.GetArrayFromImage(label)

    print(f'Image: {image_file}, Shape: {image_array.shape}')
    print(f'Label: {label_file}, Shape: {label_array.shape}')
    print('-'*50)

    if image_array.shape != label_array.shape:
        print(f'Mismatch found: {image_file} and {label_file}')
        print("x"*50)


In [43]:
print(label_list[0])

/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Dataset050_FLARE/labelsTr/FLARE_014.nii.gz


In [3]:
a="/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Dataset100_SPINE/labelsTr/SPINE_000.nii.gz"

In [12]:
image_l=sorted(glob.glob("/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/*"))
mask_l=sorted(glob.glob("/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/*"))


In [13]:
image_l

['/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0001_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0002_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0003_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0004_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0005_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0006_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0007_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0008_0000.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/d

In [17]:
mask_l

['/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0001.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0002.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0003.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0004.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0005.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0006.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0007.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0008.nii.gz',
 '/Users/kunkerdthaisong/intern_cariva/f

In [8]:
for i in range(len(mask_l)):
    
    shutil.move(mask_l[i],os.path.dirname(img_l[0]))
    shutil.move(img_l[i],os.path.dirname(mask_l[0]))

In [6]:
mask=sitk.ReadImage("/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0001.nii.gz")
mask_arr=sitk.GetArrayFromImage(mask)

print(np.unique(mask_arr))

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/volumes/FLARE22_Tr_0001.nii.gz" does not exist.

In [20]:
boo=True
#check shapes
for i in range(len(mask_l)):
    mask=sitk.ReadImage(mask_l[i])
    image=sitk.ReadImage(image_l[i])
    image_arr=sitk.GetArrayFromImage(image)
    mask_arr=sitk.GetArrayFromImage(mask)
    
    #print(np.unique(image_arr))
    print(mask_arr.shape,image_arr.shape)
    print(mask_l[i])
    if (mask_arr.shape==image_arr.shape)==False:
        boo=False
print(boo)
    
    

(110, 512, 512) (110, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0001.nii.gz
(107, 512, 512) (107, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0002.nii.gz
(101, 512, 512) (101, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0003.nii.gz
(87, 512, 512) (87, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0004.nii.gz
(85, 512, 512) (85, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0005.nii.gz
(93, 512, 512) (93, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segmentations/FLARE22_Tr_0006.nii.gz
(105, 512, 512) (105, 512, 512)
/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/segmentation/segment

In [37]:
np.unique(mask_arr)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
      dtype=uint16)

In [11]:
len(image_l),len(mask_l)


(50, 50)

In [29]:



for i in range(len(image_list)):
    shutil.move(image_list[i],os.path.dirname(label_list[0]))
    shutil.move(label_list[i],os.path.dirname(image_list[0]))

In [28]:
os.path.dirname(label_list[0])

'/Users/kunkerdthaisong/intern_cariva/flare24/datasets/nnunet_data/nnUNet_raw/Dataset050_FLARE/labelsTr'